# <center>**QDC 2025 Challenge<br>Sample-based Krylov Quantum Diagonalization**</center>

## **Overview**<br> Sampled-based Krylov Quantum Diagonalization

This challenge is based on the [_Quantum-centric Algorithm for Sampled-based Krylov Diagonalization_](https://arxiv.org/pdf/2501.09702) paper. Participants will get hands-on experience with developing an SQKD workflow and executing it on IBM Quantum hardware.

### **Abstract**
_Approximating the ground state of many-body systems is a key computational bottleneck underlying important applications in physics and chemistry. The most widely known quantum algorithm for ground state approximation, quantum phase estimation, is out of reach of current quantum processors due to its high circuit-depths. Subspace-based quantum diagonalization methods offer a viable alternative for pre- and early-fault-tolerant quantum computers. Here, we introduce a quantum diagonalization algorithm which combines two key ideas on quantum subspaces: a classical diagonalization based on quantum samples, and subspaces constructed with quantum Krylov states. We prove that our algorithm converges in polynomial time under the working assumptions of Krylov quantum diagonalization and sparseness of the ground state. We then demonstrate the scalability of our approach by performing the largest ground-state quantum simulation of impurity models using a Heron quantum processors and the Frontier supercomputer. We consider both the single-impurity Anderson model with 41 bath sites, and a system with 4 impurities and 7 bath sites per impurity. Our results are in excellent agreement with Density Matrix Renormalization Group calculations._

### **Grader metric**
Participants will be asked to reproduce experiments similar to those in the paper, aiming to achieve the best possible results compared to exact methods.

*General metrics*

- Speed, or time-to-solution.
- Quality of solution (e.g., relative error)

### **What directions are participants expected to explore?**
Investigate the impacts of noise, and how it affects bitstring quality. Determine experimental parameters, e.g., "samples per batch" required achieve some fidelity, and subspace dimensionality is needed to achieve some fidelity.

*General directions*

- Experience with the Qiskit SDK, including:

    - `AerSimulator`
    - Dynamical decoupling
    - `ibm-qiskit-runtime`
    - Pauli twirling
    - `qiskit-sqd-addon`
    - Qiskit Serverless
    - Qiskit Patterns
    - `SamplerV2`
    - Transpiler
  
- Experimentation with error suppression techniques (e.g., Pauli twirling, dynamical decoupling) through `SamplerV2` primitive
- Most effectively mapping Krylov circuits to specific device
- Qiskit transpiler via pass managers
- Optimizing post-processing

### **Computational resources required**
Estimated QPU time is $O(1)$ minute and classical Qiskit Serverless computations $O(1)$ minute for 20-30 qubits.

<a id="p0"></a>
## **Part 0**<br>Python environment setup

This challenge was developed and tested in a Python 3.12.3 virtual environment (venv) on a Macbook Pro with an M1 Max chip running macOS 15.6. Python versions 3.11.9 and 3.13.9 were also validated against.

<a id="ex0_1"></a>
### **Exercise 0.1**<br>Import helper functions and instantiate challenge grader.
Just run the following cell. This will be a first test of your environment's setup.

In [ ]:
import skqd_helpers

Load your account:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(name='qdc-2025')

Finally, instantiate the challenge grader.
Through this object, you will submit your challenge solutions.

In [ ]:
from qc_grader.challenges.qdc_2025 import qdc25_lab4
qdc25_lab4.submit_name("<TEAM_NAME>")

### **End of Part 0**
Hopefully you've made it here without any issues.

You are now ready to begin the challenge!

<a id="p1"></a>
## **Part 1**<br>Solving the perturbed transverse-field Ising model using SKQD

Consider the perturbed transverse Ising model,

$$
\begin{equation}
H = -J\sum_{j=0}^{n-2} \sigma_j^z \sigma_{j+1}^z - h_x \sum_{j=0}^{n-1} \sigma_j^x - h_z \sigma_0^z
\tag{1},
\end{equation}
$$

where $\sigma_j^i$ correspond to the Pauli matrices defined as,

$$
\begin{equation}
\sigma^x =
\begin{bmatrix}
0 & 1 \\
1 & 0 \\
\end{bmatrix},~
\sigma^y =
\begin{bmatrix}
0 & -i \\
i & 0 \\
\end{bmatrix},~
\sigma^z =
\begin{bmatrix}
1 & 0 \\
0 & -1 \\
\end{bmatrix},
\tag{2}
\end{equation}
$$

acting on qubit at index $j$, $J$ is the coupling strength between neighboring spins, and $h_{\{x,z\}}$ are strengths of the applied field strengths in the ${\{x, z\}}$ direction.

<a id="ex1_1"></a>
### **Exercise 1.1**<br>Build the Hamiltonian (10 points)
Using the provided signature below, define a method to construct the perturbed transverse-field Ising model (TFIM) Hamiltonian above as a [**`SparsePauliOp`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp).

In case you're unfamiliar with how to construct a [**`SparsePauliOp`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp) from a Hamiltonian, you can refer to the following:
  - [Quantum Simulation](https://quantum.cloud.ibm.com/learning/en/courses/utility-scale-quantum-computing/quantum-simulation#2-mapping-your-problem) (IBM Quantum Platform)
  - [Quantum Real Time Evolution using Trotterization](https://qiskit-community.github.io/qiskit-algorithms/tutorials/13_trotterQRTE.html) (qiskit-community)
  - [Coding a Hamiltonian in Qiskit](https://quantumcomputing.stackexchange.com/a/39670) (StackExchange)

In [ ]:
# Imports
from qiskit.quantum_info import SparsePauliOp
import numpy as np


def perturbed_tfim_hamiltonian(num_qubits: int, J: float, hx: float, hz: float) -> SparsePauliOp:
    """Builds the perturbed transverse-field Ising model Hamiltonian as a `SparsePauliOp`.

    Args:
        num_qubits: Number of qubits
        J: Exchange energy
        hx: Field strength in x-direction
        hz: Field strength in z-direction

    Returns:
        Hamiltonian as a SparsePauliOp following Qiskit's endian convention.
    """
    paulis = []
    coeffs = []

    return SparsePauliOp(paulis, coeffs=np.array(coeffs, dtype=np.complex64))

#### NOTE: Testing and debugging
Run the cell below to check your implementation of **`perturbed_tfim_hamiltonian()`** defined in [**Ex.1.1**](#ex1_1). There should be no errors resulting from the assertions.

In [ ]:
# Construct example Hamiltonian
test_hamiltonian = perturbed_tfim_hamiltonian(num_qubits=6, J=0.1, hx=0.2, hz=0.3)
paulis = test_hamiltonian.paulis
coeffs = test_hamiltonian.coeffs

# Expected Pauli terms
expected_paulis = ['ZZIIII', 'IZZIII', 'IIZZII', 'IIIZZI',
                    'IIIIZZ', 'XIIIII', 'IXIIII', 'IIXIII',
                    'IIIXII', 'IIIIXI', 'IIIIIX', 'ZIIIII']
# Expected Pauli coefficients
expected_coeffs = np.array([-0.1+0.j, -0.1+0.j, -0.1+0.j, -0.1+0.j,
                            -0.1+0.j, -0.2+0.j, -0.2+0.j, -0.2+0.j,
                            -0.2+0.j, -0.2+0.j, -0.2+0.j, -0.30000001+0.j])

# Checks
assert(paulis == expected_paulis)
assert(np.allclose(coeffs, expected_coeffs))


Throughout this challenge, you may want to add additional cells, such as the one above, to perform sanity checks of your own.

<div class="alert alert-block alert-success">

#### **Submit**
Run the cell below to submit your solution and compare against the reference.

In [ ]:
qdc25_lab4.grade_lab4_ex1(perturbed_tfim_hamiltonian)

<a id="ex1_2"></a>
### **Exercise 1.2**<br>Construct Krylov states (10 points)
Define the method, **`construct_krylov_circuits()`**, that constructs the Krylov (time evolution) circuits from the operator $U = \text{e}^{-iHt}$.
Implementing this operation exactly, however, generally requires an exponential number of gates.

As such, the evolution operators themselves should be of type [**`PauliEvolutionGate`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.PauliEvolutionGate). An important argument here is **`synthesis`** which implements an approximation of the unitary operator. There are a number of [**`qiskit.synthesis`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit/synthesis#module-qiskit.synthesis) methods available.

A common synthesis technique that results in low circuit depth is the Lie-Trotter product formula. This approximates the exponential of non-commuting operators with products of their exponentials up to error of second order:

$$
\begin{equation}
e^{A + B} \approx e^{A} e^{B}.
\end{equation}
$$

Feel free to experiment with different synthesis methods to reduce circuit depth.

In [ ]:
# Imports
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter

def construct_krylov_circuits(
        H_spo: SparsePauliOp, psi: QuantumCircuit, r: int, num_trotter_steps: int, dt: float
    ) -> list[QuantumCircuit]:
    """Build r Krylov circuits using time evolution generated by SparsePauliOp.

    Args:
        H_spo: Hamiltonian as SparsePauliOp
        psi: Initial state to time evolve
        r: number of Krylov basis states (i.e., Krylov dimension)
        num_trotter_steps: Number of Trotter steps per evolution gate
        dt: time step

    Returns:
        List of QuantumCircuits representing each Krylov state
    """
    circuits = []

    return circuits

<div class="alert alert-block alert-success">

#### **Submit**
Run the cell below to submit your solution and compare against the reference.
You can safely ignore any warnings of the type **`SparseEfficiencyWarning`**.

In [ ]:
qdc25_lab4.grade_lab4_ex2(construct_krylov_circuits, perturbed_tfim_hamiltonian)

<a id="ex1_3"></a>
### **Exercise 1.3**<br>Configure and execute test harness for perturbed TFIM (20 points)
Complete [**Part 1**](#part1) by working through each of the following cells. In the end, you will have found the approximate ground state energy which you will compare to the exact ground state energy.

This final exercise comprises several sub-exercises but will be graded collectively.
Your score will depend on the precision achieved relative to the result obtained via exact diagonalization.

<a id="ex1_3_1"></a>
#### **Exercise 1.3.1**<br>Construct the Hamiltonian
Set the parameters of the Hamiltonian as follows:

```python
num_qubits = 12
J = 1.0
hx = 0.1
hz = 0.1
```

We chose a system small enough system such that it can be classically simulated and diagonalized within a reasonable timeframe on a modern laptop.

In [ ]:
# Hamiltonian parameters


# Construct Hamiltonian


<a id="ex1_3_2"></a>
#### **Exercise 1.3.2**<br>Construct the initial state and Krylov circuits
The Krylov space $\mathcal{K}^r$ of order $r$ is the space spanned by the vectors obtained by multiplying increasingly higher powers of a matrix $A$, up to $r-1$, with a vector $|v\rangle$:

$$
\begin{equation}
    \mathcal{K}^r = \{ A^0 |v\rangle, A^1 |v\rangle, ..., A^{r-1} |v\rangle \}.
\end{equation}
$$

In our case, $A \equiv U = e^{-iHdt}$, in which case the Krylov space is given as,

$$
\begin{equation}
    \mathcal{K}_U^r = \{ U^0 |v\rangle, U^1 |v\rangle, ..., U^{r-1} |v\rangle \}.
\end{equation}
$$

We will start with the initial state $|v\rangle \equiv \psi = |0^n\rangle$. When generating the Krylov circuits, note that the larger Krylov dimension $r$, the better approximation to the true ground state energy can be achieved as this leads to more of the full Hilbert space will be explored.
In our case, the full Hilbert space is quite small, $2^{12} = 4096$, so a Krylov dimension $r \sim 5$ should suffice.
Also keep in mind, especially when executing jobs on real hardware, the circuit depths required for Krylov circuits with large $r$.

An important parameter to set is the **time step**, or $dt$.
A theoretical result from [**Epperly _et al._**](https://arxiv.org/abs/2110.07492) showed that an optimal value for a time step is $\pi / ||H||$.
However, in this sampling-based context, the optimal choice for the time step is a topic of ongoing study; e.g., determining the time step heuristically, one may achieve better results by scaling $\pi / ||H||$ by some $O(1)$ factor.

Additionally, as there is no exact choice of Krylov dimension to choose, you may experiment with several here using [**`AerSimulator`**](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html) before starting [**Part 2**](#p2) where you will run on real quantum hardware.

Note you may receive a harmless **`ComplexWarning`** due to a cast from imaginary to real values. This can be safely disregarded.


In [ ]:
# Calculate an optimal dt for Trotterization using the
# `dt_from_spectral_norm()` helper function.


# Specify a Krylov dimension (i.e., subspace size)


# Construct the QuantumCircuit representing the intial state


# Construct the Krylov circuits


<a id="ex1_3_3"></a>
#### **Exercise 1.3.3**<br>Set backend and transpile circuits
Select the [**`AerSimulator`**](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html) backend and use [**`generate_preset_pass_manager()`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.transpiler.generate_preset_pass_manager) to transpile the Krylov circuits to [**ISA circuits**](https://www.ibm.com/quantum/blog/isa-circuits) specific to backend.
While there are many arguments one can provide to generate a preset [**`PassManager`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.transpiler.PassManager), here we use a classical simulator so most can be ignored until [**Part 2**](#p2).



Note transpilation takes $O(1)$ seconds with an Apple M1 Max chip.

In [ ]:
# Imports


# Instantiate a backend


# Transpile Krylov circuits to ISA circuits specific for the chosen backend


<a id="ex1_3_4"></a>
#### **Exercise 1.3.4**<br>Execute the job using `SamplerV2`
As the name implies, SKQD is a sampling-based algorithm as opposed to measuring physical observables within a quantum circuit.
The reason we want to work directly with samples is for the flexibility when using post-processing techniques that leverage, e.g., certain symmetries manifest in the system under study.
These symmetries can be used to filter out or 'correct' bad bitstrings.

As such, we will use a [**`SamplerV2`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-ibm-runtime/sampler-v2) object to execute the circuits and retrieve the results.
Classical sampling with [**`AerSimulator`**](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html) takes $O(10)$ seconds with an Apple M1 Max chip for modestly sized configurations of the solution.

In [ ]:
# Imports


# Instantiate a `SamplerV2` to generate samples from the ISA circuits


# Perform classical simulation


# Retrieve results


<a id="ex1_3_5"></a>
#### **Exercise 1.3.5**<br>Get bitstrings, probabilities
Using the [**`counts_to_arrays()`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd/counts#counts_to_arrays) method from [**`qiskit-add-sqd`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd), get the bitstrings.
We will need these for projection in the next step.

In [ ]:
# Imports


# Get the bitstrings (and, optionally, probabilities)


<a id="ex1_3_6"></a>
#### **Exercise 1.3.6**<br>Calculate ground state energy
Projection requires the bitstring matrix to be sorted and ascending in order by its unsigned integer representation.
For this, [**`qiskit-add-sqd`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd)'s [**`sort_and_remove_duplicates()`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd/qubit#sort_and_remove_duplicates) can be utilized. To calculate the eigenvalues and eigenvectors, [**`solve_qubit()`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd/qubit#solve_qubit) from the same package will be useful.

In [ ]:
# Imports


# Sort bitstring matrix


# Calculate eigenvalues (and, optionally, eigenvectors)


<a id="ex1_3_7"></a>
#### **Exercise 1.3.7**<br>Ground state energy from SKQD
Finally, determine ground state energy (i.e., minimum eigenvalue).

In [ ]:
# Determine SKQD approximation to the ground state energy


<div class="alert alert-block alert-success">

#### **Submit**
Execute the following cell to check your energy estimate against the reference, calculated using exact diagonalization.
The grader for this exercise takes in sampled bitstrings and the Hamiltonian.

In [ ]:
# `bitstrings` is the variable which stores the first element returned by `counts_to_arrays()`
qdc25_lab4.grade_lab4_ex3(bitstrings, H_spo)

### **End of Part 1**
Congratulations on making it this far! Note you should have achieved an error $O(10^{-3})$ or better. If this is not the case, time permitting, go back and tweak some of the configuration (e.g., the number of Krylov basis states) to try getting a better SKQD estimate.

<a id="p2"></a>
## **Part 2**<br>SKQD workflow in a Quantum-centric Supercomputing (QCSC) Environment

In [**Part 1**](#part1), we explored how sampled-based Krylov quantum diagonalization works and hopefully learned what parameters give the highest quality basis states for diagonalization. Helper methods are provided in the **`skqd_helpers`** module that we imported in [**Part 0**](#part0).

Now, we will implement a full-fledged SKQD workflow, as illustrated in the figure below.
For this, we will be using configuration recovery -- a technique that helps one *recover* expected bitstrings from those affected by device noise -- and diagonalizing in a QCSC environment with [**Qiskit Serverless**](https://quantum.cloud.ibm.com/docs/en/guides/serverless).
The Serverless resources made available for this challenge have roughly 120GiB of useable memory to perform post-processing and diagonalization -- enough to simulate a system size of about 40 qubits.
Note that state-of-the-art experiments on 80+ qubits require terabytes of node memory, however, the experiment we will execute here is neverthess quite impressive.

Beginning in [**Exercise 2.3**](#ex2_3), we employ a [**Qiskit Pattern**](https://quantum.cloud.ibm.com/docs/en/guides/intro-to-patterns) which, as you will see, greatly simplifies quantum workflows.

<img src="./fig/qcsc-arch.png" alt="qcsc-arch"/>

Our goal in this part is to find the ground state energy of a chemistry Hamiltonian of the form,

$$
\begin{equation}
\hat{H} = \sum_{p r, \sigma} h_{pr} \hat{a}_{p \sigma} \hat{a}_{r \sigma}
            + \sum_{p r q s, \sigma \tau} \frac{(pr| qs)}{2} \hat{a}_{p \sigma}^\dagger
            \hat{a}_{q \tau}^\dagger \hat{a}_{s \tau} \hat{a}_{r \sigma},
\end{equation}
$$

where $\hat{a}_{p \sigma}^\dagger$ ($\hat{a}_{p \sigma}$) is the creation (annihilation) operator associated to the $p$-th basis set element and spin $\sigma$, and $h_{pr}$ and $(pr|qs)$ are the one- and two-body electronic integrals (see Eq. 6 in [**this paper**](https://arxiv.org/abs/2405.05068) for more details) which can be obtained from standard chemistry software, such as [**PySCF**](https://github.com/pyscf/pyscf).
You will not be required to use determine these integrals your self.

<a id="ex2_1"></a>
### **Exercise 2.1**<br>Construct SIAM Hamiltonian in momentum basis (10 points)
One of the main requirements for a successful application of the SKQD framework is that the Hamiltonian is sparse; in our case, when the bath is diagonal.
Since this is not the case in the position basis, we must rotate the orbitals in such a way as to perform a Fourier transform into the momentum basis -- where the Hamiltonian is sparse.

As the construction of this Hamiltonian is quite involved and likely requires some background in quantum chemistry, we provide helper methods **`siam_hamiltonian()`**, **`momentum_basis()`**, and **`rotated()`** in the **`skqd_helpers`** module.
You will need to consult the module and documentation to construct the SIAM Hamiltonian one- and two-body terms in the momentum basis.

In [ ]:
import numpy as np

def siam_hamiltonian_momentum(
    num_orbs: int,
    hopping: float,
    onsite: float,
    hybridization: float,
    chemical_potential: float,
) -> tuple[np.ndarray, np.ndarray]:
    """Hamiltonian for the single-impurity Anderson model (SIAM).

    Args:
        num_orbs: Number of spatial orbitals
        hopping: Hopping term
        onsite: On-site repulsion
        hybrid: Hybridization interaction
        mu: Chemical potential

    Returns:
        One- and two-body terms of Hamiltonian.
    """ 
    pass

<div class="alert alert-block alert-success">

#### **Submit**
Execute the cell below to check your implementation of the SIAM Hamiltonian.

In [ ]:
qdc25_lab4.grade_lab4_ex4(siam_hamiltonian_momentum)

<a id="ex2_2"></a>
### **Exercise 2.2**<br>Generate circuits to produce Krylov basis states (20 points)
As we did in [**Ex. 1.2**](#ex_1_2), here we also need to construct the Krylov basis.
Since we are using explicitly the one- and two-body terms representing the SIAM Hamiltonian, this construction will be slightly different; specifically, when constructing a Trotter step for the circuits.
The provided **`trotter_step()`** method in the **`skqd_helpers`** module will take care of the complexities for you.
However, time permitting, you are encouraged to check out that method and understand its workings.

For the initial state preparation, we also provide **`sqkd_helpers.prepare_initial_state()`** to  assist in completing the method definition below that will generate the Krylov circuits. 

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
import scipy

def construct_krylov_siam(
    num_orbs:int,
    impurity_index: int,
    hamiltonian: tuple[np.ndarray, np.ndarray],
    dt: float,
    krylov_dim: int
) -> list[QuantumCircuit]:
    """Generate Krylov circuits for SIAM.

    Args:
        num_orbs: Number of spatial orbitals
        hamiltonian: one- and two-body Hamiltonian terms
        dt: Time step
        krylov_dim: Number of Krylov basis states
    
    Returns:
        SIAM Krylov circuits.
    """
    circuits = []

    return circuits

<div class="alert alert-block alert-success">

#### **Submit**
Check your implementation and receive your score by running the cell below.

In [ ]:
qdc25_lab4.grade_lab4_ex5(construct_krylov_siam, siam_hamiltonian_momentum)

<a id="ex2_3"></a>
### **Exercise 2.3**<br>Using a Qiskit Pattern to solve the SIAM Hamiltonian (30 points)
For this exercise, consider a half-filled system with a single impurity and the following parameters:

```python
num_orbs = 24
hybridization = 1.0
hopping = 1.0
onsite = 1.0
chemical_potential = -0.5 * onsite
```

Note $20 \leq n_{\text{orbs}} \leq 24$. Although we are using Qiskit Serverless, a QCSC-like environment, it is not yet a replacement for supercomputing platforms (e.g., ORNL's Frontier, NERSC's Perlmutter, etc.).

As in [**Exercise 1.1**](#ex1_3), this exercise comprises multiple parts, and will be graded once all parts are completed.
Your score will depend on the precision achieved relative to the DMRG calculation.

<a id="ex2_3_1"></a>
#### **Exercise 2.3.1**<br>Map the problem to quantum circuits and operators
The SKQD workflow requires that there is polynomially overlap of the initial state with the true ground state.
However, while it is not always possible to know the true ground state in advance, there are useful ways that can help us get close.
In our case, we use [**Givens rotations**]() to prepare an initial state that is known to approximate the ground state of our model in the free fermion (i.e., non-interacting) model of our Hamiltonian.

<div style="text-align: center;">
    <img src="./fig/initial-state.png" alt="initial-state" width="25%"/>
</div>

Despite our Hamiltonian involving interaction terms, this _ansatz_ serves well.

Using a combination of the methods defined in the previous exercises, as well as helper method **`prepare_initial_state()`**, construct circuits representing the Krylov basis states.
You can obtain a good time step using **`skqd_helpers.dt_from_spectral_norm()`** as before, but must choose a Krylov dimension appropriately.

In [ ]:
# Set Hamiltonian parameters


# Construct SIAM Hamiltonian


# Construct Krylov circuits


<a id="ex2_3_2"></a>
#### **Exercise 2.3.2**<br>Optimize for target hardware
In [**Part 1**](#p1), we focused on classical simulation using [**`AerSimulator`**](https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.AerSimulator.html).
To execute a circuit on real quantum hardware, one must transpile the abstract circuit to the quantum instruction set architecture ([**ISA**](https://www.ibm.com/quantum/blog/isa-circuits)) for that device.
This ensure the circuit is represented in such a way that the backend hardware can understand, i.e., in the form of its native basis gates.

Select your backend and construct Krylov [**ISA circuits**](https://www.ibm.com/quantum/blog/isa-circuits).

In [ ]:
# Imports


# Retrieve backend


# Construct optimized ISA circuits for backend


<a id="ex2_3_3"></a>
#### **Exercise 2.3.3**<br>Execute on target hardware
Now that we have all the necessary ingredients, we are ready to execute the circuits on quantum hardware.

As before, use the Qiskit [**`SamplerV2`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-ibm-runtime/sampler-v2) primitive to generate samples from the [**ISA circuits**](https://www.ibm.com/quantum/blog/isa-circuits). Feel free to break this exercise up into several cells.

In [ ]:
# Imports


# Instantiate `SamplerV2` and set options
# Be sure to set SamplerV2.options.max_execution_time = 600 to stay within
# your allocation. Otherwise, your job may be terminated before completion.


# Sample circuits


# Combine the counts from individual Trotter (Krylov) circuits


<a id="ex2_3_4"></a>
#### **Exercise 2.3.4**<br>Post-process results
A key component to the SQKD workflow is post-processing, where bitstrings and their counts are used to perform **configuration recovery**: a technique that serves to mitigate (loosely, 'correct') errors introduced during circuit execution and state measurement.
The configuration recovery introduced in the paper, [**_Chemistry Beyond the Scale of Exact Diagonalization on a Quantum-centric Supercomputer_**](https://arxiv.org/abs/2405.05068) (beginning on page 16) gives the full details of a 'self-consistent' way to correct noisy bitstrings.
For our purposes, the figure below gives a high-level overview of how the algorithm works.
<div style="text-align: center;">
    <img src="./fig/config-recovery.png" alt="configuration-recovery" width="35%"/>
</div>

In words, the algorithm proceeds as follows:
1. Input is a list of noisy sampled, $\tilde{\mathcal{X}} = \{ \mathbf{x} | \sim \tilde{P}_\Psi(\mathbf{x}) \}$, generated by a noisy quantum processor.
2. If particle number (a symmetry, or conservation, of our Hamiltonian) is not conserved, i.e., $N_\mathbf{x} \neq N$, $|N_\mathbf{x} - N|$ bits are flipped probabilistically to generate a new set $\tilde{\mathcal{X}}_R$ of _recovered configurations_.
3. Build $K$ batches of $d$ configurations, $\mathcal{S}^{(1)}, ..., \mathcal{S}^{(K)}$, from $\tilde{\mathcal{X}}_R$, distributed according to the frequencies of $\mathbf{x} \in \tilde{\mathcal{X}}_R$.
4. Obtain ground state energies are computed from $\hat{H}_{\mathcal{S}^{(K)}}$ which are then used to calculate updated occupancies.
5. Repeat until convergence.

Early versions of [**`qiskit-addon-sqd`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd) required users implement much of this algorithm programmatically; thankfully, we now have a super straightforward way to run this entire part of the workflow -- configuration recovery, diagonalization, and calculate ground state energies -- with a single method call to [**`diagonalize_fermionic_hamiltonian()`**](https://quantum.cloud.ibm.com/docs/en/api/qiskit-addon-sqd/fermion#diagonalize_fermionic_hamiltonian).

To perform post-processing, we employ Qiskit Serverless for sufficient classical resources to handle the scale of our problem.
Using the template below, configure an instance of [**Qiskit Serverless**](https://quantum.cloud.ibm.com/docs/en/guides/serverless).
Provided for you is the engine to perform configuration recovery and diagonalization in the **`serverless`** directory -- you only need to correctly set up a client.
To help guide you, please refer to the section [**Deploy to IBM Quantum Platform**](https://quantum.cloud.ibm.com/docs/en/guides/serverless-first-program#deploy-to-ibm-quantum-platform) of the [**Qiskit Serverless**](https://quantum.cloud.ibm.com/docs/en/guides/serverless) documentation.

In [ ]:
# Imports
from qiskit_ibm_catalog import QiskitServerless

# Instantiate a `QsikitServerless` object
client = QiskitServerless(name='qdc-2025')

# We need to upload the program intended to run to the Serverless cloud environment.
# (And re-upload it any time we change it's source code!)


We are now ready to run the post-processing workflow.
This can be done either 'locally' on the system running **`jupyter-notebook`** or 'remotely' within the [**Qiskit Serverless**](https://quantum.cloud.ibm.com/docs/en/guides/serverless) environment set up above using the **`skqd_helpers.classically_diagonalize()`** method.

The following cell provides a basic configuration to execute the workflow.
Note setting **`local = True`** is a good debugging exercise, though should only be used for systems less than 30 qubits.
Additionally, you may want to experiment with different values of **`num_batches`** and **`samples_per_batch`** to achieve a better approximation to the ground state obtained via density matrix renormalization (DMRG).

In [ ]:
# `True` to run locally, `False` to run in Serverless environment
local = False

# SQD Options
energy_tol = 1e-4
occupancies_tol = 1e-3
max_iterations = 3  # Feel free to modify

# Eigenstate solver options
num_batches = 3  # Feel free to modify
samples_per_batch = 100  # Feel free to modify
max_cycle = 200
symmetrize_spin = True
carryover_threshold = 1e-5

When you're ready, make your call to **``skqd_helpers.classically_diagonalize()``** in the code cell below.

In [ ]:
# Call `skqd_helpers.classically_diagonalize()` and store the results


<div class="alert alert-block alert-success">

#### **Submit**
Check how your SKQD workflow fares against DMRG by running the cell below.

In [ ]:
qdc25_lab4.grade_lab4_ex6(result, num_orbs, hopping, onsite, hybridization, chemical_potential)

### **End of Part 2**
If you have time to, continue experimenting and try improving your score.
However, note that in the event of a tie, the winner is determined by submission date; i.e., the quicker you submit, the better.

# **End of SKQD Challenge**
Congratulations on completing the SKQD Challenge!
Thank you for your participation!